In [1]:
import matplotlib
matplotlib.use("qt4agg")
import matplotlib.pyplot as plt

import numpy as np

from astropy.io import fits
import skimage
from skimage import io
import astrobf
from astrobf.Shin import img_scale

/home/hoseung/anaconda3/envs/tm39/lib/python3.9/importlib/__init__.py:127: MatplotlibDeprecationWarning: 
The matplotlib.backends.backend_qt4agg backend was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
  return _bootstrap._gcd_import(name[level:], package, level)


In [7]:
ddir = '../example_data/'
outdir = ddir
fn = ddir+"J000311.00+155754.0-r.fits"
sig_fract = 5.0
percent_fract = 0.01

hdulist = fits.open(fn)
img_header = hdulist[0].header
img_data_raw = hdulist[0].data
hdulist.close()
width=img_data_raw.shape[0]
height=img_data_raw.shape[1]
print("#INFO : ", fn, width, height)
img_data_raw = np.array(img_data_raw, dtype=float)
#sky, num_iter = img_scale.sky_median_sig_clip(img_data, sig_fract, percent_fract, max_iter=100)
sky, num_iter = img_scale.sky_mean_sig_clip(img_data_raw, sig_fract, percent_fract, max_iter=10)
print("sky = ", sky, '(', num_iter, ')')
img_data = img_data_raw - sky
min_val = 0.0
print("... min. and max. value : ", np.min(img_data), np.max(img_data))

#INFO :  ../example_data/J000311.00+155754.0-r.fits 181 181
sky =  0.0517539047625 ( 9 )
... min. and max. value :  -0.142209600749 92.2954369522


### skimage 

Because I found it cumbersome to get the right number of pixels in matplotlib. 

In [8]:
def scale_to_unity(img):
    return (img-img.min())/img.ptp()

def save_fig(fname, img, cmap=None, origin='lower'):
    """
    NOTE
    ----
        Use np.fliplr for left-right flipping
        Or, np.flip with axis arguement for general flipping.
    """
    if cmap is not None: img = cmap(img)
    if origin == 'lower': img = np.flipud(img)
    io.imsave(fname, img)

In [9]:
cmap = None
#cm = plt.cm.get_cmap('inferno')
new_img = img_scale.sqrt(img_data, scale_min = min_val)

save_fig(outdir+'sqrt.png', skimage.util.img_as_ubyte(scale_to_unity(new_img)), cmap=cmap) # convert to uit or you will get a warning.

new_img = img_scale.power(img_data, power_index=3.0, scale_min = min_val)
save_fig(outdir+'power.png', skimage.util.img_as_ubyte(scale_to_unity(new_img)), cmap=cmap)

new_img = img_scale.log(img_data, scale_min = min_val)
save_fig(outdir+'log.png', skimage.util.img_as_ubyte(scale_to_unity(new_img)), cmap=cmap)

new_img = img_scale.linear(img_data, scale_min = min_val)
save_fig(outdir+'linear.png', skimage.util.img_as_ubyte(scale_to_unity(new_img)), cmap=cmap)

new_img = img_scale.asinh(img_data, scale_min = min_val, non_linear=0.01)
save_fig(outdir+'asinh_beta_01.png', skimage.util.img_as_ubyte(scale_to_unity(new_img)), cmap=cmap)

new_img = img_scale.asinh(img_data, scale_min = min_val, non_linear=0.5)
save_fig(outdir+'asinh_beta_05.png', skimage.util.img_as_ubyte(scale_to_unity(new_img)), cmap=cmap)

new_img = img_scale.asinh(img_data, scale_min = min_val, non_linear=2.0)
save_fig(outdir+'asinh_beta_20.png', skimage.util.img_as_ubyte(scale_to_unity(new_img)), cmap=cmap)

new_img = img_scale.histeq(img_data_raw, num_bins=256)
save_fig(outdir+'histeq.png', skimage.util.img_as_ubyte(scale_to_unity(new_img)), cmap=cmap)

new_img = img_scale.logistic(img_data_raw, center = 0.03, slope = 0.3)
save_fig(outdir+'logistic.png', skimage.util.img_as_ubyte(scale_to_unity(new_img)), cmap=cmap)

img_scale : sqrt
img_scale : power
img_scale : log
img_scale : linear
img_scale : asinh
img_scale : asinh
img_scale : asinh
img_scale : logistic


<ipython-input-8-048be3dfa34a>:13: UserWarning: ../example_data/power.png is a low contrast image
  io.imsave(fname, img)
<ipython-input-8-048be3dfa34a>:13: UserWarning: ../example_data/linear.png is a low contrast image
  io.imsave(fname, img)


## Matplotlib version

#### Note on matplotlib colormaps

Only few colormaps are linear (perceptually uniform) in lightness: *viridis, plasma, inferno, magma, cividis* 
Among them, only **inferno** and **magma** ranges (0,100) lightness. 

*binary, gist_yarg, gist_gray, gray* also cover the whole lightness range (but are NOT perfectly linear)

In [10]:
def save_image_fix_dpi(fname, data, dpi=100, cmap='inferno'):
    """
    save ndarray as an image with exact number of pixels.
    """
    shape=np.shape(data)[0:2][::-1]
    size = [float(i)/dpi for i in shape]
    fig = plt.figure()
    fig.set_size_inches(size)
    ax = plt.Axes(fig,[0,0,1,1])
    ax.set_axis_off()
    fig.add_axes(ax)
    ax.imshow(data, origin='lower', cmap=cmap)
    fig.savefig(fname, dpi=dpi)
    plt.close()

In [11]:
cmap = 'hot'
new_img = img_scale.sqrt(img_data, scale_min = min_val)
save_image_fix_dpi(outdir+'sqrt_plt.png', new_img, cmap=cmap)

new_img = img_scale.power(img_data, power_index=3.0, scale_min = min_val)
save_image_fix_dpi(outdir+'power_plt.png', new_img, cmap=cmap)

new_img = img_scale.log(img_data, scale_min = min_val)
save_image_fix_dpi(outdir+'log_plt.png', new_img, cmap=cmap)

new_img = img_scale.linear(img_data, scale_min = min_val)
save_image_fix_dpi(outdir+'linear_plt.png', new_img, cmap=cmap)

new_img = img_scale.asinh(img_data, scale_min = min_val, non_linear=0.01)
save_image_fix_dpi(outdir+'asinh_beta_01_plt.png', new_img, cmap=cmap)

new_img = img_scale.asinh(img_data, scale_min = min_val, non_linear=0.5)
save_image_fix_dpi(outdir+'asinh_beta_05_plt.png', new_img, cmap=cmap)

new_img = img_scale.asinh(img_data, scale_min = min_val, non_linear=2.0)
save_image_fix_dpi(outdir+'asinh_beta_20_plt.png', new_img, cmap=cmap)

new_img = img_scale.histeq(img_data_raw, num_bins=256)
save_image_fix_dpi(outdir+'histeq_plt.png', new_img, cmap=cmap)

new_img = img_scale.logistic(img_data_raw, center = 0.03, slope = 0.3)
save_image_fix_dpi(outdir+'logistic_plt.png', new_img, cmap=cmap)

img_scale : sqrt
img_scale : power
img_scale : log
img_scale : linear
img_scale : asinh
img_scale : asinh
img_scale : asinh
img_scale : logistic
